**Next method**

In [1]:
import sys
print(sys.executable)

c:\Users\edward_b\OneDrive - Institute for Fiscal Studies\Work\Brazil social insurance\venv\Scripts\python.exe


***Pyautogui***

In [2]:
import pyautogui
import os
import time
import subprocess
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from bs4 import BeautifulSoup
import psutil
from selenium.webdriver.support.ui import Select
import matplotlib.pyplot as plt
import logging
from logging.handlers import RotatingFileHandler
import random
import requests
import re

In [3]:
os.chdir("C:/Users/edward_b/OneDrive - Institute for Fiscal Studies/Work/Brazil social insurance")

**Define functinos**

In [4]:
# Data storage
data = []

# Function to kill Chrome processes
def kill_chrome():
    """Kill all Chrome processes."""
    for proc in psutil.process_iter(['pid', 'name']):
        if 'chrome' in proc.info['name'].lower():
            try:
                proc.kill()
            except psutil.NoSuchProcess:
                pass

# Scraper function
def scrape_data(cnpj,year, soup, table):
    if not table:
        '''
        print(f"No table found for {cnpj}")
        data.append({'cnpj': cnpj,
                      'periodo': year,
                      'data_found': False})
        return
        '''
    rows = table.find('tbody').find_all('tr', class_='pa')
    for row in rows:
        cells = row.find_all('td')
        if len(cells) < 10:
            continue
        data.append({
            'cnpj': cnpj,
            'periodo': cells[1].get_text(strip=True),
            'apurado': cells[2].get_text(strip=True),
            'INSS': cells[3].get_text(strip=True),
            'principal': cells[4].get_text(strip=True),
            'multa': cells[5].get_text(strip=True),
            'juros': cells[6].get_text(strip=True),
            'total': cells[7].get_text(strip=True),
            'vencimento': cells[8].get_text(strip=True),
            'acolhimento': cells[9].get_text(strip=True),
            'data_found': True
        })

**Import master**

In [ ]:
path = "raw/CNPJ numbers"
cnpj_master = pd.read_csv(f'{path}/simples.csv', sep=',', encoding='utf-8')
cnpj_master = cnpj_master[['cnpj_basico','opcao_mei']]

# find length of cnpj
cnpj_master["length cnpj_basico"] = cnpj_master["cnpj_basico"].astype(str).str.len()
pd.crosstab(cnpj_master["length cnpj_basico"], cnpj_master["opcao_mei"], margins=True, margins_name="Total")

C:\Users\edward_b\AppData\Local\Temp\ipykernel_17524\2426474582.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  cnpj_master = pd.read_csv(f'{path}/simples.csv', sep=',', encoding='utf-8')


In [ ]:
cnpj_master["cnpj_basico"] = cnpj_master["cnpj_basico"].astype(str)
cnpj_master = cnpj_master[cnpj_master['opcao_mei'] == 1]
cnpj_master.drop(columns=['length cnpj_basico'], inplace=True)

**Import example full file**

In [63]:
cnpj_test = pd.read_csv(f'{path}/establishmentsPI.csv', sep=',', encoding='utf-8')
#cnpj_test = cnpj_test[['cnpj']]
cnpj_test["cnpj_basico"] = cnpj_test["cnpj"].astype(str).str[:8]

C:\Users\edward_b\AppData\Local\Temp\ipykernel_17524\1643610931.py:1: DtypeWarning: Columns (12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  cnpj_test = pd.read_csv(f'{path}/establishmentsPI.csv', sep=',', encoding='utf-8')


In [ ]:
cnpj_merged = pd.merge(cnpj_master, cnpj_test, left_on='cnpj_basico', right_on='cnpj_basico', how='inner')
cnpj_merged["cnpj"] = cnpj_merged["cnpj"].astype(str)
#cnpj_merged["cnpj"].str.len().hist()

In [67]:
cnpj_merged = cnpj_merged[cnpj_merged['cnpj'].str.len() == 14]

#Check for duplicates
duplicates = cnpj_merged[cnpj_merged['cnpj'].duplicated(keep=False)]
print(len(duplicates) == 0)
cnpj_merged = cnpj_merged.drop_duplicates(subset=['cnpj'], keep='first')

True


In [ ]:
cnpj_merged.to_csv('MEI_numbers.csv', sep=',', encoding='utf-8', index=False)

*****-----------------------Load in MEI numbers and start scraping-----------------------*****

In [5]:
cnpj_merged = pd.read_csv('MEI_numbers.csv', sep=',', encoding='utf-8')

In [ ]:
# proxy_list = pd.read_csv("scripts/proxies.txt", sep=',', encoding='utf-8', header=None, names=['proxy'])
# proxy_list = proxy_list["proxy"].to_list()
# proxy_list

In [6]:
# get random sample of 10
cnpj_merged = cnpj_merged.sample(n=100, random_state=10)
#convert cnpj's to a list
cnpj_list = cnpj_merged['cnpj'].tolist()
cnpj_merged.shape # 13,894 obs

(100, 3)

**Check if proxies work**

In [ ]:


# proxy = "66.201.7.151"  # Replace with the proxy you want to test

# try:
#     response = requests.get(
#         "https://httpbin.org/ip",
#         proxies={"http": proxy, "https": proxy},
#         timeout=10
#     )
#     print("Proxy is working. IP seen by server:", response.text)
# except Exception as e:
#     print("Proxy failed:", e)

Nothing done.
Proxy failed: HTTPSConnectionPool(host='httpbin.org', port=443): Max retries exceeded with url: /ip (Caused by ProxyError('Unable to connect to proxy', NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001F05BD1BFB0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it')))


**Set up scraper**

In [ ]:
cnpj_list = [str(i) for i in cnpj_list]
cnpj_list

In [7]:
cnpj_list = ["35184782000140"]

In [8]:
import shutil
import os

chrome_profile_path = "C:/Temp/ChromeDebug"
if os.path.exists(chrome_profile_path):
    shutil.rmtree(chrome_profile_path)  # delete the profile folder

In [9]:
log_file = 'mei_scraper_log.log'
with open(log_file, 'w'):
    pass  # This empties the file
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        RotatingFileHandler(log_file, maxBytes=5*1024*1024, backupCount=3),  # 5MB per file
        logging.StreamHandler()  # Optional: print to console too
    ]
)

In [20]:
# ---- Define url and cnpj list ----
url = "https://www8.receita.fazenda.gov.br/SimplesNacional/Aplicacoes/ATSPO/pgmei.app/Identificacao"
data = []
timings = []
total_start_time = time.time()


for cnpj in cnpj_list:
    #proxy = random.choice(proxy_list)
    logging.info("Processing CNPJ:", cnpj)
    start_time = time.time()

    try:

        # ---- Step 1: Start Chrome in remote debug mode ----
        subprocess.Popen([
            r"C:/Program Files/Google/Chrome/Application/chrome.exe",
            #f"--proxy-server={proxy}",
            "--remote-debugging-port=9222",
            "--user-data-dir=" + chrome_profile_path,
            "--start-maximized",  # or "--start-fullscreen"
            "--disable-popup-blocking",  # optional, disable for debugging only
            "--disable-extensions",
            "--no-first-run",
            "--no-default-browser-check"
        ])
        time.sleep(2)  # Give Chrome time to launch

        # ---- Step 2: Use pyautogui to interact with the site ----
        pyautogui.hotkey('ctrl', 'l')
        pyautogui.typewrite(url, interval=0.01)
        pyautogui.press('enter')  
        time.sleep(2)

        pyautogui.moveTo(x=722, y=391, duration=1)
        pyautogui.click()
        pyautogui.typewrite(cnpj, interval=0.1)

        pyautogui.moveTo(x=722, y=514, duration=1)
        pyautogui.click()
        time.sleep(2)

        options = Options()
        options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
        #options.add_argument(f'--proxy-server={proxy}')
        driver = webdriver.Chrome(options=options)
        wait = WebDriverWait(driver, 3)

        driver.get("https://www8.receita.fazenda.gov.br/SimplesNacional/Aplicacoes/ATSPO/pgmei.app/emissao")
        time.sleep(1.5)

        # First try: Bootstrap-styled dropdown
        try:
            dropdown_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'button[data-id="anoCalendarioSelect"]')))
            dropdown_button.click()
            time.sleep(1)

            year_elements = wait.until(EC.presence_of_all_elements_located(
                (By.CSS_SELECTOR, ".dropdown-menu.inner li a span.text")
            ))
            enabled_years = [el.text.strip() for el in year_elements if el.text.strip()]
            # remove elements from the list that contain "Não optante"
            enabled_years = [year for year in enabled_years if "Não optante" not in year]

            # Raise an exception if no enabled years are found
            if not enabled_years:
                raise ValueError("No enabled years found in the dropdown menu.")

            print("Bootstrap dropdown enabled years for CNPJ ", cnpj , ":", enabled_years)
            use_bootstrap = True
            

        except Exception as e:
            print("Bootstrap dropdown failed, falling back to native <select> method.")
            # Try native <select>
            select_element = wait.until(EC.presence_of_element_located((By.ID, "anoCalendarioSelect")))
            dropdown = Select(select_element)
            enabled_years = [o.text.strip() for o in dropdown.options if o.text.strip()]
            enabled_years = [year for year in enabled_years if "Não optante" not in year]
            print("Native <select> enabled years for CNPJ ", cnpj,":", enabled_years)
            use_bootstrap = False

        print("scraping years", enabled_years)
        #enabled_years = [str(max(enabled_years))]
        enabled_years.insert(0, "2010")  #add a year to the start of the list
        
        for index, year in enumerate(enabled_years):
            try:
                if use_bootstrap:
                    dropdown_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'button[data-id="anoCalendarioSelect"]')))
                    driver.execute_script("arguments[0].click();", dropdown_button)
                    time.sleep(1.5)  # allow dropdown to render
                    
                    print("Clicking on year:", year)
                    year_option = wait.until(EC.element_to_be_clickable(
                        (By.XPATH, f"//span[@class='text' and normalize-space(text())='{year}']")
                    ))
                    time.sleep(2)
                    driver.execute_script("arguments[0].click();", year_option)
                    #ActionChains(driver).move_to_element(year_option).click().perform()
                    print(f"Selected (Bootstrap) year: {year}")
                else:
                    dropdown = Select(driver.find_element(By.ID, "anoCalendarioSelect"))
                    dropdown.select_by_visible_text(year)
                    print(f"Selected (native) year: {year}")

                ok_button = driver.find_element(By.CSS_SELECTOR, "button[type='submit']")
                ok_button.click()
                time.sleep(2)
                
                # check if table is present
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                table = soup.find('table', class_='table table-hover table-condensed emissao is-detailed')
                if not table:
                    print(f"No table found for {cnpj} in year {year}. Skipping the rest.")
                    # Mark all remaining years as not found
                    for remaining_year in enabled_years[index:]:
                        data.append({
                            'cnpj': cnpj,
                            'periodo': remaining_year,
                            'data_found': False
                        })
                    break  # Exit the year loop
                    
                scrape_data(cnpj,year, soup, table)

                driver.back()
                time.sleep(2)

            except Exception as e:
                print(f"Error with year {year}:", e)

    except Exception as outer_error:
        print(f"Fatal error with CNPJ {cnpj}:", outer_error)

    finally:
        try:
            driver.quit()
        except:
            pass
        kill_chrome()
        end_time = time.time()  # ⏱ End timer
        elapsed = end_time - start_time
        timings.append(elapsed)
        total_elapsed = time.time() - total_start_time
        average_elapsed = sum(timings) / len(timings)
        
        logging.info(f"Finished CNPJ: {cnpj} in {elapsed:.2f} seconds\n")
        logging.info(f"Average time per CNPJ: {average_elapsed:.2f} seconds")
        logging.info(f"Total time elapsed: {total_elapsed:.2f} seconds\n")
        time.sleep(2)

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 73, in emit
    if self.shouldRollover(record):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 197, in shouldRollover
    msg = "%s\n" % self.format(record)
                   ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 703, in format
    record.message = record.getMessage()
                     ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\Python

Bootstrap dropdown enabled years for CNPJ  28740844000198 : ['2020', '2021', '2022', '2023', '2024', '2025']
scraping years ['2020', '2021', '2022', '2023', '2024', '2025']
Clicking on year: 2010
Error with year 2010: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6FE72CF45+75717]
	GetHandleVerifier [0x00007FF6FE72CFA0+75808]
	(No symbol) [0x00007FF6FE4F8F9A]
	(No symbol) [0x00007FF6FE54F4C6]
	(No symbol) [0x00007FF6FE54F77C]
	(No symbol) [0x00007FF6FE5A2577]
	(No symbol) [0x00007FF6FE5773BF]
	(No symbol) [0x00007FF6FE59F39C]
	(No symbol) [0x00007FF6FE577153]
	(No symbol) [0x00007FF6FE540421]
	(No symbol) [0x00007FF6FE5411B3]
	GetHandleVerifier [0x00007FF6FEA2D71D+3223453]
	GetHandleVerifier [0x00007FF6FEA27CC2+3200322]
	GetHandleVerifier [0x00007FF6FEA45AF3+3322739]
	GetHandleVerifier [0x00007FF6FE746A1A+180890]
	GetHandleVerifier [0x00007FF6FE74E11F+211359]
	GetHandleVerifier [0x00007FF6FE735294+109332]
	GetHandleVerifier [0x00007FF6FE735442+109762]
	GetHandleVerifier [0x00007FF6

2025-05-24 12:56:59,946 - INFO - Finished CNPJ: 28740844000198 in 36.36 seconds

2025-05-24 12:56:59,948 - INFO - Average time per CNPJ: 36.36 seconds
2025-05-24 12:56:59,949 - INFO - Total time elapsed: 36.37 seconds

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 73, in emit
    if self.shouldRollover(record):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 197, in shouldRollover
    msg = "%s\n" % self.format(record)
                   ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFounda

Bootstrap dropdown enabled years for CNPJ  11757216000112 : ['2020', '2021', '2022', '2023', '2024', '2025']
scraping years ['2020', '2021', '2022', '2023', '2024', '2025']
Clicking on year: 2010
Error with year 2010: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6FE72CF45+75717]
	GetHandleVerifier [0x00007FF6FE72CFA0+75808]
	(No symbol) [0x00007FF6FE4F8F9A]
	(No symbol) [0x00007FF6FE54F4C6]
	(No symbol) [0x00007FF6FE54F77C]
	(No symbol) [0x00007FF6FE5A2577]
	(No symbol) [0x00007FF6FE5773BF]
	(No symbol) [0x00007FF6FE59F39C]
	(No symbol) [0x00007FF6FE577153]
	(No symbol) [0x00007FF6FE540421]
	(No symbol) [0x00007FF6FE5411B3]
	GetHandleVerifier [0x00007FF6FEA2D71D+3223453]
	GetHandleVerifier [0x00007FF6FEA27CC2+3200322]
	GetHandleVerifier [0x00007FF6FEA45AF3+3322739]
	GetHandleVerifier [0x00007FF6FE746A1A+180890]
	GetHandleVerifier [0x00007FF6FE74E11F+211359]
	GetHandleVerifier [0x00007FF6FE735294+109332]
	GetHandleVerifier [0x00007FF6FE735442+109762]
	GetHandleVerifier [0x00007FF6

2025-05-24 12:58:15,860 - INFO - Finished CNPJ: 11757216000112 in 73.89 seconds

2025-05-24 12:58:15,860 - INFO - Average time per CNPJ: 55.12 seconds
2025-05-24 12:58:15,868 - INFO - Total time elapsed: 112.28 seconds

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 73, in emit
    if self.shouldRollover(record):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 197, in shouldRollover
    msg = "%s\n" % self.format(record)
                   ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFound

...

2025-05-24 14:27:24,225 - INFO - Finished CNPJ: 13927044000103 in 74.19 seconds

2025-05-24 14:27:24,225 - INFO - Average time per CNPJ: 53.73 seconds
2025-05-24 14:27:24,225 - INFO - Total time elapsed: 5460.64 seconds

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 73, in emit
    if self.shouldRollover(record):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 197, in shouldRollover
    msg = "%s\n" % self.format(record)
                   ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoun

Bootstrap dropdown enabled years for CNPJ  39848945000101 : ['2020', '2021', '2022', '2023', '2024', '2025']
scraping years ['2020', '2021', '2022', '2023', '2024', '2025']
Clicking on year: 2010
Error with year 2010: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6FE72CF45+75717]
	GetHandleVerifier [0x00007FF6FE72CFA0+75808]
	(No symbol) [0x00007FF6FE4F8F9A]
	(No symbol) [0x00007FF6FE54F4C6]
	(No symbol) [0x00007FF6FE54F77C]
	(No symbol) [0x00007FF6FE5A2577]
	(No symbol) [0x00007FF6FE5773BF]
	(No symbol) [0x00007FF6FE59F39C]
	(No symbol) [0x00007FF6FE577153]
	(No symbol) [0x00007FF6FE540421]
	(No symbol) [0x00007FF6FE5411B3]
	GetHandleVerifier [0x00007FF6FEA2D71D+3223453]
	GetHandleVerifier [0x00007FF6FEA27CC2+3200322]
	GetHandleVerifier [0x00007FF6FEA45AF3+3322739]
	GetHandleVerifier [0x00007FF6FE746A1A+180890]
	GetHandleVerifier [0x00007FF6FE74E11F+211359]
	GetHandleVerifier [0x00007FF6FE735294+109332]
	GetHandleVerifier [0x00007FF6FE735442+109762]
	GetHandleVerifier [0x00007FF6

2025-05-24 14:28:39,440 - INFO - Finished CNPJ: 39848945000101 in 73.20 seconds

2025-05-24 14:28:39,440 - INFO - Average time per CNPJ: 53.92 seconds
2025-05-24 14:28:39,440 - INFO - Total time elapsed: 5535.86 seconds

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 73, in emit
    if self.shouldRollover(record):
       ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\handlers.py", line 197, in shouldRollover
    msg = "%s\n" % self.format(record)
                   ^^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.12_3.12.2800.0_x64__qbz5n2kfra8p0\Lib\logging\__init__.py", line 999, in format
    return fmt.format(record)
           ^^^^^^^^^^^^^^^^^^
  File "C:\Program Files\WindowsApps\PythonSoftwareFoun

Bootstrap dropdown enabled years for CNPJ  43280648000171 : ['2021', '2022', '2023', '2024', '2025']
scraping years ['2021', '2022', '2023', '2024', '2025']
Clicking on year: 2010
Error with year 2010: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6FE72CF45+75717]
	GetHandleVerifier [0x00007FF6FE72CFA0+75808]
	(No symbol) [0x00007FF6FE4F8F9A]
	(No symbol) [0x00007FF6FE54F4C6]
	(No symbol) [0x00007FF6FE54F77C]
	(No symbol) [0x00007FF6FE5A2577]
	(No symbol) [0x00007FF6FE5773BF]
	(No symbol) [0x00007FF6FE59F39C]
	(No symbol) [0x00007FF6FE577153]
	(No symbol) [0x00007FF6FE540421]
	(No symbol) [0x00007FF6FE5411B3]
	GetHandleVerifier [0x00007FF6FEA2D71D+3223453]
	GetHandleVerifier [0x00007FF6FEA27CC2+3200322]
	GetHandleVerifier [0x00007FF6FEA45AF3+3322739]
	GetHandleVerifier [0x00007FF6FE746A1A+180890]
	GetHandleVerifier [0x00007FF6FE74E11F+211359]
	GetHandleVerifier [0x00007FF6FE735294+109332]
	GetHandleVerifier [0x00007FF6FE735442+109762]
	GetHandleVerifier [0x00007FF6FE71BA59+4825]
	

2025-05-24 14:29:18,643 - INFO - Finished CNPJ: 43280648000171 in 37.19 seconds

2025-05-24 14:29:18,645 - INFO - Average time per CNPJ: 53.76 seconds
2025-05-24 14:29:18,645 - INFO - Total time elapsed: 5575.06 seconds



In [21]:
df = pd.DataFrame(data)
df

,cnpj,periodo,apurado,INSS,principal,multa,juros,total,vencimento,acolhimento,data_found
0,28740844000198,Janeiro/2020,Sim,,-,-,-,-,-,-,True
1,28740844000198,Fevereiro/2020,Sim,,-,-,-,-,-,-,True
2,28740844000198,Março/2020,Sim,,-,-,-,-,-,-,True
3,28740844000198,Abril/2020,Sim,,"R$ 57,25","R$ 11,45","R$ 25,16","R$ 93,86",23/11/2020,26/05/2025,True
4,28740844000198,Maio/2020,Sim,,"R$ 57,25","R$ 11,45","R$ 25,07","R$ 93,77",21/12/2020,26/05/2025,True
...,...,...,...,...,...,...,...,...,...,...,...
4210,43280648000171,Dezembro/2021,Não,,,"R$ 56,00","R$ 11,20","R$ 21,68","R$ 88,88",20/01/2022,True
4211,43280648000171,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4212,43280648000171,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4213,43280648000171,2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [22]:
pd.DataFrame(data).to_csv('MEI_data.csv', sep=',', encoding='utf-8', index=False)